# 1. Problem Definition & Objective
- **Selected Project Track:** Smart Cultural Storyteller.
- **Problem Statement:** Design a tool that uses AI to retell folk tales, history, or traditions in engaging formats—interactive text, audio, or visuals.
- **Real-world relevance and motivation** By automating the generation of cultural narratives and accompanying media, we provide a scalable way for educators and cultural enthusiasts to preserve oral traditions.

## 2. Data Understanding & Preparation
In this project, the data is dynamic rather than static. 
* **Dataset source:** We use a "Synthetic/API-based" approach where data is generated in real-time based on user prompts.
* **Data loading:** Inputs are collected via the Streamlit interface (Culture, Tone, and Length).
* **Cleaning & Preprocessing:** The system cleans the LLM output by splitting the narrative from the "Moral" and applies line limits to ensure the story is concise.

In [1]:
# ============================================================
# Imports & Configuration
# ============================================================

import os
import random
import streamlit as st
import requests
from PIL import Image, ImageDraw, ImageFont
import io
import textwrap
import imageio
import subprocess
import imageio_ffmpeg
import pyttsx3
import tempfile
import google.generativeai as genai
import numpy as np
import wave
import contextlib
from groq import Groq


c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\hp\AppData\Local\Temp\ipykernel_4140\239035701.py:17: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [2]:
# ============================================================
# DEBUG FLAG
# ============================================================

DEBUG_IMAGE_MODE = True

# ============================================================
# Page Configuration
# ============================================================

st.set_page_config(
    page_title="Smart Cultural Storyteller",
    page_icon="📖",
    layout="wide"
)

st.markdown("## 📖 Smart Cultural Storyteller")

if DEBUG_IMAGE_MODE:
    st.warning("🧪 DEBUG MODE: Offline text enabled to save API quota.")


2026-01-14 22:07:44.910 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:44.911 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:46.278 
  command:

    streamlit run C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2026-01-14 22:07:46.280 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:46.282 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:46.283 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:46.283 Thread 'MainThread': missing ScriptRunContext! This warning can be i

In [3]:
# ============================================================
# API Keys
# ============================================================

GROQ_API_KEY = os.getenv("GROQ_API_KEY", "").strip()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "").strip()

# ============================================================
# Clients
# ============================================================

groq_client = Groq(api_key=GROQ_API_KEY) if GROQ_API_KEY else None

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    GEMINI_MODEL = genai.GenerativeModel("models/gemini-flash-latest")
else:
    GEMINI_MODEL = None


## 3. Model/System Design
This section outlines the AI architecture and the logic behind choosing specific techniques.

* **AI Technique used:** Hybrid approach using LLMs (Large Language Models) for text, API-based Image Generation, and TTS (Text-to-Speech).
* **Architecture explanation:** The system follows a linear pipeline: 1. User Input -> 2. LLM Narrative Generation -> 3. Image Generation -> 4. Audio Synthesis -> 5. Video Compilation.
* **Justification:** We use a fallback logic (Groq -> Gemini -> Offline) to ensure the system is "fail-safe," providing a result even if the internet or API keys are unavailable.

## 4. Core Implementation
This section details the primary logic for AI model inference and the multi-media pipeline.
* **Model Inference Logic:** Using direct API calls to Groq and if API call fails due to some reason then it will direct the call to Gemini to generate cultural narratives and if Gemini call also fails then story will be generated locally in the offline mode.
* **Prompt Engineering:** We use structured instructions to ensure the AI follows specific cultural tones and line limits.
* **Pipeline:** Integration of text-to-speech and visual generation to create a cohesive story experience.

In [4]:
# ============================================================
# CORE IMPLEMENTATION: LLM LOGIC
# ============================================================

def generate_with_groq(prompt: str) -> str:
    if not groq_client:
        raise RuntimeError("Groq API key not set")
    chat = groq_client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=400,
    )
    return chat.choices[0].message.content.strip()


def generate_with_gemini(prompt: str) -> str:
    if not GEMINI_MODEL:
        raise RuntimeError("Gemini API key not set")
    return GEMINI_MODEL.generate_content(prompt).text.strip()


def generate_story_with_fallback(prompt: str, offline_fn, *offline_args):
    if DEBUG_IMAGE_MODE:
        text, _ = offline_fn(*offline_args)
        return text, "Offline generator (debug)"

    try:
        return generate_with_groq(prompt), "Groq API"
    except:
        pass
    try:
        return generate_with_gemini(prompt), "Gemini API"
    except:
        pass

    text, _ = offline_fn(*offline_args)
    return text, "Offline generator"


STORY_LINE_LIMITS = {"Short": 8, "Medium": 14, "Long": 20}


In [5]:
# ============================================================
# Offline Story Logic
# ============================================================
def offline_story_opening(culture_name, tone_name):
    if not culture_name:
        culture_name = "a faraway land"

    openings = [
        f"Long ago, in the land of {culture_name}, there was a small village where everyone knew each other's names.",
        f"In a forgotten corner of {culture_name}, near forests and rivers, lived a child with a heart full of questions.",
        f"On the edge of {culture_name}, where mountains touched the clouds, there stood a lonely house with a strange rumour."
    ]
    incidents = [
        " One evening, a tired traveller arrived with a mysterious box that no one was allowed to open.",
        " One day, the village well suddenly dried up, and fear spread through the streets.",
        " One night, a bright light appeared in the sky, and only the bravest soul dared to follow it."
    ]

    intro = random.choice(openings) + random.choice(incidents)
    return intro, ["Be brave and investigate", "Stay safe and avoid trouble"]


def offline_story_ending(choice_text, culture_name, length_name, tone_name):
    if "brave" in choice_text.lower():
        main = "The hero chose courage, stepping forward when others stepped back and discovering hidden strength."
    else:
        main = "The hero chose safety, finding a calm and wise path that protected everyone."

    moral = random.choice([
        "True courage can be quiet and gentle.",
        "Patience and kindness solve many problems.",
        "Wisdom lies in choosing the right moment.",
        "Helping others brings lasting happiness."
    ])

    return f"{main}\n\nMoral: {moral}", moral


def offline_full_story(culture_name, tone_name):
    intro, _ = offline_story_opening(culture_name, tone_name)
    choice = random.choice(["Be brave and investigate", "Stay safe and avoid trouble"])
    ending, moral = offline_story_ending(choice, culture_name, "", tone_name)
    return f"{intro}\n\n{ending}", moral


In [6]:
# ============================================================
# Image Providers (STABLE + SAFE)
# ============================================================

def generate_image_with_pollinations(prompt: str) -> bytes:
    """
    Attempts to generate image using Pollinations.
    Raises exception if anything goes wrong.
    """
    url = "https://image.pollinations.ai/prompt/" + requests.utils.quote(prompt)
    resp = requests.get(url, timeout=6)

    if resp.status_code != 200:
        raise RuntimeError("Pollinations API failed")

    if "image" not in resp.headers.get("Content-Type", ""):
        raise RuntimeError("Invalid image response")

    return resp.content


def generate_image_offline(prompt: str) -> bytes:
    """
    Simple offline placeholder image.
    (Keeps your original behavior intact)
    """
    img = Image.new("RGB", (512, 512), color=(235, 225, 200))
    draw = ImageDraw.Draw(img)
    draw.text((80, 240), "Offline Image", fill=(80, 60, 40))
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()


def generate_image_with_fallback(prompt: str):
    """
    Safe image generator with fallback.
    Contract (DO NOT CHANGE):
        returns (image_bytes, source_name)
    """

    # -------- Try Pollinations first --------
    try:
        url = "https://image.pollinations.ai/prompt/" + requests.utils.quote(prompt)

        # Short timeout prevents UI freeze
        resp = requests.get(url, timeout=6)

        # Status must be OK
        if resp.status_code != 200:
            raise RuntimeError("Pollinations returned non-200")

        # Response must be an image
        if "image" not in resp.headers.get("Content-Type", ""):
            raise RuntimeError("Pollinations did not return image")

        # Validate image bytes using PIL
        img = Image.open(io.BytesIO(resp.content))
        img.verify()  # raises exception if corrupted

        # If all checks pass → return image
        return resp.content, "Pollinations.ai"

    except Exception:
        # -------- Offline fallback (guaranteed) --------
        img = Image.new("RGB", (512, 512), color=(235, 225, 200))
        draw = ImageDraw.Draw(img)
        draw.text((80, 240), "Offline Image", fill=(80, 60, 40))

        buf = io.BytesIO()
        img.save(buf, format="PNG")

        return buf.getvalue(), "Offline placeholder"

In [7]:
# ============================================================
# 🔊 TTS
# ============================================================

def get_available_voices():
    engine = pyttsx3.init()
    return engine.getProperty('voices')


def text_to_speech(text: str, voice_id: str, rate: int) -> str:
    engine = pyttsx3.init()
    engine.setProperty('rate', rate)
    if voice_id:
        engine.setProperty('voice', voice_id)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
    path = temp_file.name
    temp_file.close()

    engine.save_to_file(text, path)
    engine.runAndWait()
    return path

In [8]:
# ============================================================
# 🔇 Silent audio helper (for image pauses)
# ============================================================

def create_silent_audio(duration_sec, output_path):
    import wave, struct
    fr = 44100
    frames = int(duration_sec * fr)

    with wave.open(output_path, 'w') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(fr)
        wf.writeframes(struct.pack('<h', 0) * frames)

    return output_path


In [9]:
# ============================================================
# Whiteboard Frame Generator (Text Typing Effect)
# ============================================================

def generate_whiteboard_scenes_with_images(scenes, images_bytes_list, fps=15):
    width, height = 1280, 720
    bg_color = (245, 245, 245)
    text_color = (20, 20, 20)
    font_size = 36

    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except:
        font = ImageFont.load_default()

    all_frames = []

    for idx, scene_text in enumerate(scenes):
        base = Image.new("RGB", (width, height), bg_color)
        base_draw = ImageDraw.Draw(base)

        x, y = 60, 60
        line_height = font_size + 14

        lines = []
        for para in scene_text.split("\n"):
            lines.extend(textwrap.wrap(para, 60))
            lines.append("")

        for line in lines:
            typed = ""
            for ch in line:
                typed += ch
                frame = base.copy()
                draw = ImageDraw.Draw(frame)
                draw.text((x, y), typed, fill=text_color, font=font)
                all_frames.append(frame)

            base_draw.text((x, y), line, fill=text_color, font=font)
            y += line_height
            all_frames.extend([base.copy()] * int(fps * 0.3))

            if y > height - 100:
                all_frames.extend([base.copy()] * int(fps * 0.8))
                base = Image.new("RGB", (width, height), bg_color)
                base_draw = ImageDraw.Draw(base)
                y = 60

        all_frames.extend([base.copy()] * int(fps * 1.0))

        if idx < len(images_bytes_list) and images_bytes_list[idx]:
            img = Image.open(io.BytesIO(images_bytes_list[idx])).convert("RGB")
            img = img.resize((width, height))
            all_frames.extend([img.copy()] * int(fps * 2))

    return all_frames


In [10]:
# ============================================================
# Audio Duration Helper
# ============================================================

def get_audio_duration(audio_path):
    with contextlib.closing(wave.open(audio_path, 'r')) as f:
        frames = f.getnframes()
        rate = f.getframerate()
        return frames / float(rate)


In [11]:
# ============================================================
# Save Video with Audio (FFmpeg)
# ============================================================

def save_video_with_audio(frames, audio_path, output_path="story_video.mp4", fps=15):
    temp_video = "temp_video.mp4"

    audio_duration = get_audio_duration(audio_path)
    required_frames = int(audio_duration * fps) + 1
    frames = frames[:required_frames]

    imageio.mimsave(temp_video, frames, fps=fps)

    ffmpeg_path = imageio_ffmpeg.get_ffmpeg_exe()

    cmd = [
        ffmpeg_path, "-y",
        "-i", temp_video,
        "-i", audio_path,
        "-c:v", "copy",
        "-c:a", "aac",
        output_path
    ]

    try:
        subprocess.run(cmd, check=True)
    except Exception:
        os.replace(temp_video, output_path)
        return output_path

    os.remove(temp_video)
    return output_path


In [12]:
# ============================================================
# SAFE Session State Initialization
# ============================================================

DEFAULT_SESSION_STATE = {
    "stage": "idle",
    "intro": "",
    "ending_text": "",
    "moral": "",
    "image_bytes": None,
    "last_api_status": "",
    "image_autogenerated": False,
    "auto_generate_video": True,
    "selected_voice_id": None
}

for k, v in DEFAULT_SESSION_STATE.items():
    if k not in st.session_state:
        st.session_state[k] = v


# ============================================================
# Streamlit UI
# ============================================================

left, right = st.columns([1.1, 2])

with left:
    culture = st.text_input("🌍 Cultural theme", "Indian Folk Tale")
    tone = st.selectbox("🎭 Tone", ["Moral", "Funny", "Adventurous", "Emotional"])
    story_length = st.selectbox("📏 Story length", ["Short", "Medium", "Long"], index=1)

    voices = get_available_voices()
    names = [v.name for v in voices]
    name = st.selectbox("🗣 Voice", names)
    st.session_state.selected_voice_id = voices[names.index(name)].id

    start_clicked = st.button("✨ Start / Restart Story", use_container_width=True)


with right:
    if start_clicked:
        max_lines = STORY_LINE_LIMITS[story_length]

        prompt = (
            f"Write a complete folk tale inspired by {culture}. "
            f"Tone: {tone}. "
            f"The story must be concise and no longer than {max_lines} short lines. "
            f"End with a clear moral prefixed by 'Moral:'."
        )

        story_text, source = generate_story_with_fallback(
            prompt, offline_full_story, culture, tone
        )

        if "Moral:" in story_text:
            p = story_text.split("Moral:")
            st.session_state.intro = p[0].strip()
            st.session_state.moral = p[1].strip()
        else:
            st.session_state.intro = story_text.strip()
            st.session_state.moral = ""

        st.session_state.stage = "end"
        st.session_state.auto_generate_video = True
        st.session_state.last_api_status = f"Generated using: {source}"
        st.rerun()

    # ========================================================
    # END STAGE DISPLAY + VIDEO GENERATION  (F PART)
    # ========================================================

    if st.session_state.stage == "end":

        st.subheader("📖 Story")
        st.write(st.session_state.intro)

        if st.session_state.moral:
            st.info(f"Moral: {st.session_state.moral}")

        # Auto-generate image once
        if not st.session_state.image_autogenerated:
            img, src = generate_image_with_fallback(st.session_state.intro)
            st.session_state.image_bytes = img
            st.session_state.image_autogenerated = True

        if st.session_state.image_bytes:
            st.image(st.session_state.image_bytes)

        # ---------------- FULL STORY VIDEO ----------------
        st.markdown("### 🎥 Full Story Video")

        if st.session_state.auto_generate_video:
            st.session_state.auto_generate_video = False

            scenes = [st.session_state.intro]
            if st.session_state.moral:
                scenes.append(f"Moral: {st.session_state.moral}")

            segment_videos = []

            with st.spinner("Creating synced story video..."):
                for i, scene in enumerate(scenes):

                    audio = text_to_speech(
                        scene,
                        st.session_state.selected_voice_id,
                        170
                    )

                    frames = generate_whiteboard_scenes_with_images([scene], [None])
                    seg = f"segment_{i}.mp4"
                    save_video_with_audio(frames, audio, seg)
                    segment_videos.append(seg)

                    # image pause after first scene
                    if i == 0 and st.session_state.image_bytes:
                        img = Image.open(io.BytesIO(st.session_state.image_bytes)).convert("RGB")
                        img = img.resize((1280, 720))
                        fps = 15
                        pause = 3
                        img_frames = [img.copy()] * int(pause * fps)
                        silent = create_silent_audio(pause, f"silence_{i}.wav")
                        img_seg = f"image_pause_{i}.mp4"
                        save_video_with_audio(img_frames, silent, img_seg)
                        segment_videos.append(img_seg)

            # Concatenate all video segments
            with open("segments.txt", "w") as f:
                for v in segment_videos:
                    f.write(f"file '{v}'\n")

            ffmpeg = imageio_ffmpeg.get_ffmpeg_exe()
            subprocess.run([
                ffmpeg, "-y",
                "-f", "concat", "-safe", "0",
                "-i", "segments.txt",
                "-c:v", "libx264",
                "-c:a", "aac",
                "-movflags", "+faststart",
                "story_video.mp4"
            ], check=True)

            with open("story_video.mp4", "rb") as f:
                video_bytes = f.read()

            st.video(video_bytes)
            st.download_button(
                "📥 Download Story Video",
                video_bytes,
                "story_video.mp4",
                "video/mp4"
            )


2026-01-14 22:07:47.408 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2026-01-14 22:07:47.414 Session state does not function when running a script without `streamlit run`
2026-01-14 22:07:47.417 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:47.419 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:47.420 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:47.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:47.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:47.427 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07:47.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 22:07

## 5. Evaluation & Analysis
This section assesses the quality and performance of the storyteller.
* **Metrics:** We use qualitative evaluation based on narrative flow, cultural accuracy, and the synchronization of audio-visual elements.
* **Sample Outputs:** The system successfully generates a story, an image, and a synced video for themes like "Indian Folk Tale" or "African Legend."
* **Performance & Limitations:** While the AI is creative, it may sometimes produce generic "folk" tropes.The API call may also fail sometimes,the offline fallback ensures the tool is always usable.

## 6. Ethical Considerations & Responsible AI
* **Bias & Fairness:** AI models can sometimes reflect stereotypes. We encourage users to be specific in their "Cultural theme" to guide the AI toward more accurate representation.
* **Responsible Use:** This tool is designed for education and entertainment. All media is labeled as AI-generated to maintain transparency.

## 7. Conclusion & Future Scope
* **Summary:** The Smart Cultural Storyteller successfully integrates LLMs, Image APIs, and TTS to transform simple cultural prompts into immersive multimedia folk tales. 
* **Future Scope:**
    1. Multi-language support to tell stories in native regional dialects.
    2. Fine-tuning models on specific historical archives for better factual accuracy.
    3. Adding interactive "Choose your own adventure" branching paths within the video.